In [1]:
import pandas as pd
import numpy as np
import altair as alt
from tqdm import tqdm
import graphviz
alt.data_transformers.enable('csv')

DataTransformerRegistry.enable('csv')

In [2]:
df1 = pd.read_csv(r"C:\Users\Administrator\Repositories\GreenSecurity-FirstExperiment\Dell Inc. Latitude Latitude 7430 Windows 10\No Scan\Power Saver Plan\Measurement 4\processes_data.csv")

In [3]:
splunk_processes = df1.dropna()[df1.dropna()['PNAME'].str.contains('splunkd')]['PID'].unique()
splunk_processes

array([13560,  7072,  5436], dtype=int64)

In [4]:
df_splunk = df1[df1['PID']==13560]
# df_splunk_forwarder = df1[df1['PID']==18704]
# pid_dict = {17768:'splunk',18704:'splunk_forwarder'}
# df_splunk_all = df1[df1['PID'].isin([17768,18704])]
# df_splunk_all['proc_name'] = df_splunk_all['PID'].apply(lambda x: pid_dict[x])
df_splunk_all = df_splunk

In [7]:
logs = pd.read_csv('../../Downloads/logs_msr_4.csv')
logs._time = pd.to_datetime(logs._time)
web_logs = logs[logs['source']=='webserver'].dropna(axis=1)
user_logs = logs[logs['source']=='user'].dropna(axis=1)

In [8]:
df_splunk_all['time'] = logs._time.min()+pd.to_timedelta(df_splunk_all['Time(sec)'],unit='s')-pd.to_timedelta(3,unit='s')
df_splunk_all['time']

C:\Users\Administrator\AppData\Local\Temp\ipykernel_25808\3798526432.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_splunk_all['time'] = logs._time.min()+pd.to_timedelta(df_splunk_all['Time(sec)'],unit='s')-pd.to_timedelta(3,unit='s')


2       2023-01-19 15:39:27.435000181+02:00
229     2023-01-19 15:39:33.323798656+02:00
458     2023-01-19 15:39:38.933315992+02:00
686     2023-01-19 15:39:44.186336040+02:00
912     2023-01-19 15:39:49.943782091+02:00
                        ...                
21230   2023-01-19 15:48:50.814589977+02:00
21453   2023-01-19 15:48:56.562582254+02:00
21674   2023-01-19 15:49:03.827366829+02:00
21896   2023-01-19 15:49:10.177335978+02:00
22119   2023-01-19 15:49:16.756515980+02:00
Name: time, Length: 100, dtype: datetime64[ns, pytz.FixedOffset(120)]

In [10]:
web_logs._raw.to_list()

['2023-01-19T15:49:41.686751 Handled incoming Web Request with sessionID=6d9ad76f-d713-4181-aa0a-310b33a6217a and sending Database Request with connectionID=2ecf7c40-cb3c-442f-ae43-6db43a34af67 query=UPDATE  EMP SET SAL  = 0.5 * SAL;',
 '2023-01-19T15:49:41.686751 Handled incoming Web Request with sessionID=3421688b-8993-4d44-88f1-4cd239d26c96 and sending Database Request with connectionID=30906269-f7a0-497e-a51e-1f5c269152b4 query=SELECT PLAYERS.PLAYERNO, NAME, NUMBER_OF_PENALTIES, NUMBER_OF_TEAMS FROM   PLAYERS,  (SELECT PLAYERNO, COUNT(*) AS NUMBER_OF_PENALTIES FROM   PENALTIES GROUP BY PLAYERNO) AS NUMBER_PENALTIES,  (SELECT PLAYERNO, COUNT(*) AS NUMBER_OF_TEAMS FROM   TEAMS GROUP BY PLAYERNO) AS NUMBER_TEAMS WHERE  PLAYERS.PLAYERNO = NUMBER_PENALTIES.PLAYERNO AND  PLAYERS.PLAYERNO = NUMBER_TEAMS.PLAYERNO;',
 '2023-01-19T15:49:41.686751 Database Replied with status=down connectionID=6028884f-ff05-4c83-921c-9d709cced5eb',
 '2023-01-19T15:49:41.686751 Database Replied with status=dow

In [136]:
cpu = alt.Chart(df_splunk_all).mark_line(
    ).encode(
    # alt.Color("proc_name:N"),
    alt.Y2("CPU(%):Q"),
    alt.X("time:T"),
).properties(title='CPU usage')

web_logs_count = alt.Chart(web_logs).mark_bar(
    ).encode(
    alt.Y("count():Q"),
    alt.X("_time:T"),
)


In [137]:
web_logs_count+cpu

c:\Users\Administrator\.conda\envs\py310\lib\site-packages\altair\utils\core.py:283: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():
c:\Users\Administrator\.conda\envs\py310\lib\site-packages\altair\utils\core.py:283: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [10]:
alt.Chart(df_splunk_all).mark_line(
    ).encode(
    alt.Color("proc_name:N"),
    alt.Y("WRITE_IO(#):Q", stack=None),
    alt.X("Time(sec):Q"),
).properties(title='WRITE_IO num')

alt.Chart(...)

In [11]:
alt.Chart(df_splunk_all).mark_line(
    ).encode(
    alt.Color("proc_name:N"),
    alt.Y("READ_IO(#):Q", stack=None),
    alt.X("Time(sec):Q"),
).properties(title='READ_IO num')

alt.Chart(...)

In [12]:
alt.Chart(df_splunk_all).mark_line(
    ).encode(
    alt.Color("proc_name:N"),
    alt.Y("READ_IO(KB):Q", stack=None),
    alt.X("Time(sec):Q"),
).properties(title='READ_IO kb')

alt.Chart(...)

In [13]:
alt.Chart(df_splunk_all).mark_line(
    ).encode(
    alt.Color("proc_name:N"),
    alt.Y("WRITE_IO(KB):Q", stack=None),
    alt.X("Time(sec):Q"),
).properties(title='WRITE_IO kb')

alt.Chart(...)

In [114]:
import sweetviz as sv

my_report = sv.analyze(logs)
my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"

c:\Users\Administrator\.conda\envs\py310\lib\site-packages\sweetviz\dataframe_report.py:74: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  all_source_names = [cur_name for cur_name, cur_series in source_df.iteritems()]
c:\Users\Administrator\.conda\envs\py310\lib\site-packages\sweetviz\dataframe_report.py:109: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  filtered_series_names_in_source = [cur_name for cur_name, cur_series in source_df.iteritems()


                                             |          | [  0%]   00:00 -> (? left)

c:\Users\Administrator\.conda\envs\py310\lib\site-packages\sweetviz\series_analyzer_cat.py:28: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in category_counts.iteritems():
c:\Users\Administrator\.conda\envs\py310\lib\site-packages\sweetviz\series_analyzer_cat.py:28: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in category_counts.iteritems():
c:\Users\Administrator\.conda\envs\py310\lib\site-packages\sweetviz\series_analyzer_cat.py:28: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in category_counts.iteritems():
c:\Users\Administrator\.conda\envs\py310\lib\site-packages\sweetviz\series_analyzer_cat.py:28: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in category_counts.iteritems():
c:\Users\Administrator\.conda\envs\py310\lib\sit

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Create Processes tree

In [2]:
import psutil
def rec_print_proc(proc, i=0):
    tabs = ''.join(i*['\t\t'])
    print(f"\n{tabs}{proc.__str__()}")
    print(f"\n{tabs}num of childs: {len(proc.children())}")
    # print(f"\n{tabs}parents: {proc.parent().__str__()}")
    print(f"\n{tabs}cmdline: {proc.cmdline()}\n\n")
    if not proc.children():
        return
    for child in proc.children():
        rec_print_proc(child, i+1)

for proc in psutil.process_iter(['pid', 'name', 'username']):
    if proc.name().__contains__('splunk'):
        if proc.parent().name() == 'services.exe':
            rec_print_proc(proc)
            print('\n\n')


psutil.Process(pid=17768, name='splunkd.exe', status='running', started='09:45:54')

num of childs: 5

cmdline: ['C:\\Program Files\\Splunk\\bin\\splunkd.exe', 'service']



		psutil.Process(pid=15320, name='conhost.exe', status='running', started='09:45:54')

		num of childs: 0

		cmdline: ['\\??\\C:\\windows\\system32\\conhost.exe', '0x4']



		psutil.Process(pid=15088, name='mongod.exe', status='running', started='09:46:19')

		num of childs: 0

		cmdline: ['mongod.exe', '--dbpath=C:\\Program Files\\Splunk\\var\\lib\\splunk\\kvstore\\mongo', '--storageEngine=wiredTiger', '--wiredTigerCacheSizeGB=2.250000', '--port=9003', '--timeStampFormat=iso8601-utc', '--oplogSize=200', '--keyFile=C:\\Program Files\\Splunk\\var\\lib\\splunk\\kvstore\\mongo\\splunk.key', '--setParameter=enableLocalhostAuthBypass=0', '--setParameter=oplogFetcherSteadyStateMaxFetcherRestarts=0', '--replSet=3F8CB569-37ED-42BD-A150-5C6ED48C612A', '--bind_ip=0.0.0.0', '--sslMode=requireSSL', '--sslAllowInvalidHostnames